[`pidgin.weave`](weave.md.ipynb) allows code to be execute with the display function.

In [1]:
    import ast, jinja2, sys, nbconvert, nbformat, contextlib, emoji, collections, IPython, mistune, doctest, yaml, re, traitlets, textwrap, types, typing as t, importlib, importnb,  jinja2, builtins, jsonschema
    with __import__('importnb').Notebook():
        from . import tangle
        
    with tangle.Pidgin(): 
        from . import shell
        from .specifications import template, markdown, yaml_, transform_cell, emojis

In [2]:
    if __name__ == '__main__': 
        %reload_ext pidgin

In [3]:
    def load_ipython_extension(shell: IPython.InteractiveShell):
        from . import testing, template
        document, test = map(make_module, '_document_ _test_'.split())
        test = shell.test = testing.Doctest(user_module=test, parent=shell, parents=[shell])
        weave = shell.weave = Weave(user_module=document, parent=shell, parents=[test, shell])
        template = shell.template = template.Template(parent=shell, parents=[weave, test, shell])
        shell.display_formatter.formatters['text/html'].for_type(str, format_http)
        shell.events.register('post_run_cell', test.run_cell)
        shell.events.register('post_run_cell', weave.run_cell)
        shell.enable_html_pager = True        

In [4]:
    def format_http(str): 
        if str.startswith(('http:', 'https:')): return mistune.markdown(str)

In [5]:
    class Weave(template.Template, markdown.Markdown, yaml_.FrontMatter, shell.Shell):
        test = traitlets.Instance(IPython.InteractiveShell)
        weave = traitlets.Instance(IPython.InteractiveShell)
        template = traitlets.Instance(IPython.InteractiveShell)
        
        def __init__(Weave, *args, **kwargs):
            markdown.Markdown.__init__(Weave, *args, **kwargs)
            shell.Shell.__init__(Weave, *args, **kwargs)
            yaml_.FrontMatter.__init__(Weave)
            
        def run_cell(Weave, text: str, *args, **kwargs):
            if isinstance(text, IPython.core.interactiveshell.ExecutionResult): text = text.info.raw_cell
            publish = do_publish(''.join(text))
            text = text.lstrip('//')
            if text in Weave.user_ns:
                data, metadata = Weave.parent.display_formatter.format(Weave.user_ns[text])
                Weave.parent.display_pub.publish(data, metadata, transient={'raw': True})
                return 
            text, metadata = Weave.front_matter(text)
            
            text = emojis.emojize(text)
            text = template.Template.render(Weave, text, **metadata)
            
            if len(text.splitlines()) == 1 and text.startswith(('http:', 'https:')):
                data, metadata = Weave.parent.display_formatter.format(IPython.display.IFrame(text, "100%", "600px", metadata=metadata))
            else:
                text = markdown.Markdown.render(Weave, text)
                if publish:
                    data, metadata = Weave.parent.display_formatter.format(IPython.display.HTML(text, metadata=metadata))
                else: data = {}
            
            (data or metadata) and Weave.parent.display_pub.publish(data, metadata)


In [6]:
    def do_publish(str) -> bool:
        display = bool(str.partition('\n')[0].strip()) & (not str.startswith('\\'))
        source = transform_cell.markdown_to_python(str)
        display = display & (not bool(only_contains_block_code(str, source)))
        try:
            if str.strip() != ast.literal_eval(source.rstrip().rstrip(';')).strip(): display = False
        finally: return display

In [7]:
    def make_module(str):
        try:  return importlib.import_module(str)
        except:  return sys.modules.__setitem__(str, types.ModuleType(str)) or sys.modules[str]

In [8]:
    def only_contains_block_code(raw: str, source: str) -> bool: 
        return raw.startswith('%%') or list(map(str.rstrip, textwrap.dedent(raw).splitlines())) == list(map(str.rstrip, textwrap.dedent(source).splitlines()))

In [9]:
    def unload_ipython_extension(shell: IPython.InteractiveShell):
        ...

In [10]:
    def load_ipython_extension(shell: IPython.InteractiveShell):
        from . import testing, template
        document, test = map(make_module, '_document_ _test_'.split())
        test = shell.test = testing.Doctest(user_module=test, parent=shell, parents=[shell])
        weave = shell.weave = Weave(user_module=document, parent=shell, parents=[test, shell])
        template = shell.template = template.Template(parent=shell, parents=[weave, test, shell])
        shell.display_formatter.formatters['text/html'].for_type(str, format_http)
        shell.events.register('post_run_cell', test.run_cell)
        shell.events.register('post_run_cell', weave.run_cell)
        shell.enable_html_pager = True        